# Оценка парсера мероприятий (Event Miner Agent)

Этот ноутбук:
1. Парсит сообщения из Telegram каналов
2. Извлекает события с помощью Event Miner Agent
3. Сохраняет пары сообщение-событие в CSV
4. Оценивает качество парсинга через 3 агентов-оценщиков
5. Агрегирует и выводит финальные результаты

## Инструкции по запуску

1. Убедитесь, что установлены все зависимости
2. Создайте файл сессии Telegram (при первом запуске потребуется авторизация)
3. Добавьте реальные каналы в список `channels`


## 1. Настройка и импорты


In [1]:
import sys
import os
from pathlib import Path

# Добавляем корневую директорию проекта в путь
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")


Project root: /Users/andreisuhov/Desktop/llm/Journey_agent


In [2]:
import json
import pandas as pd
import numpy as np
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from tqdm import tqdm
from datetime import datetime
import warnings
import asyncio

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ResourceWarning)

# Для работы с async в Jupyter
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    print("⚠️ nest_asyncio не установлен. Установите: pip install nest_asyncio")

# Загружаем переменные окружения
from dotenv import load_dotenv
load_dotenv(project_root / ".env")

# OpenAI
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"OpenAI API Key: {'✓ загружен' if OPENAI_API_KEY else '✗ НЕ НАЙДЕН'}")

# Инициализация OpenAI клиента
client = OpenAI(api_key=OPENAI_API_KEY)


OpenAI API Key: ✓ загружен


In [3]:
# Импорт парсера Telegram и Event Miner Agent
from src.sync_worker.tg_parser import TelegramParser
from src.sync_worker.event_miner_agent import EventMinerAgent, EventExtractor
from src.models.event import Event

print("✓ TelegramParser импортирован")
print("✓ EventMinerAgent импортирован")
print("✓ Event модель импортирована")


✓ TelegramParser импортирован
✓ EventMinerAgent импортирован
✓ Event модель импортирована


## 2. Настройка каналов для парсинга


In [4]:
# Список каналов для парсинга
# ВАЖНО: Замените мок-каналы на реальные!

channels = [
    # Мок-каналы (замените на реальные)
    # "https://t.me/afisha",
    # "https://t.me/DavaiSNami",
    # "https://t.me/meropriatia1",
    "@prekrasnie_spb"
]

# Количество сообщений для парсинга из каждого канала
MESSAGES_PER_CHANNEL = 20

print(f"📋 Настроено {len(channels)} каналов для парсинга")
print(f"📨 Будет спарсено по {MESSAGES_PER_CHANNEL} сообщений из каждого канала")
print(f"📊 Всего ожидается: {len(channels) * MESSAGES_PER_CHANNEL} сообщений")


📋 Настроено 1 каналов для парсинга
📨 Будет спарсено по 20 сообщений из каждого канала
📊 Всего ожидается: 20 сообщений


## 3. Парсинг сообщений из каналов


In [5]:
async def parse_channels(channels: List[str], limit: int = 20) -> List[Dict]:
    """
    Парсит сообщения из списка каналов.
    
    Args:
        channels: Список каналов (URL или username)
        limit: Количество сообщений из каждого канала
    
    Returns:
        Список словарей с сообщениями
    """
    all_messages = []
    
    async with TelegramParser() as parser:
        # Запускаем клиент
        await parser.client.start()
        
        for channel in tqdm(channels, desc="Парсинг каналов"):
            try:
                print(f"\n📥 Парсинг канала: {channel}")
                
                messages = await parser.get_channel_messages(
                    channel=channel,
                    limit=limit
                )
                
                for msg in messages:
                    if msg.message:  # Только сообщения с текстом
                        all_messages.append({
                            'id': msg.id,
                            'date': msg.date.isoformat() if msg.date else None,
                            'text': msg.message,
                            'channel': channel
                        })
                
                print(f"   ✓ Получено {len(messages)} сообщений (с текстом: {sum(1 for m in messages if m.message)})")
                
            except Exception as e:
                print(f"   ✗ Ошибка при парсинге {channel}: {e}")
                continue
    
    return all_messages

print("Функция парсинга каналов готова")


Функция парсинга каналов готова


In [6]:
# Парсим сообщения из каналов
print("🔄 Начинаем парсинг каналов...\n")

parsed_messages = asyncio.get_event_loop().run_until_complete(
    parse_channels(channels, limit=MESSAGES_PER_CHANNEL)
)

print(f"\n✓ Всего спарсено {len(parsed_messages)} сообщений с текстом")


🔄 Начинаем парсинг каналов...



Парсинг каналов: 100%|██████████| 1/1 [00:00<00:00,  6.06it/s]


📥 Парсинг канала: @prekrasnie_spb
   ✓ Получено 20 сообщений (с текстом: 15)

✓ Всего спарсено 15 сообщений с текстом


In [7]:
# Показываем примеры сообщений
print("📝 Примеры спарсенных сообщений:\n")
for i, msg in enumerate(parsed_messages[:3], 1):
    print(f"{i}. [{msg['date'][:10] if msg['date'] else 'N/A'}] Канал: {msg['channel']}")
    print(f"   Текст: {msg['text'][:150]}..." if len(msg['text']) > 150 else f"   Текст: {msg['text']}")
    print()


📝 Примеры спарсенных сообщений:

1. [2025-12-26] Канал: @prekrasnie_spb
   Текст: 🗓 27 декабря 17:00–21:00
💳 BIG ECSTATIC SANTA BEAT с Vankavanka 

В самом конце декабря, когда город живёт ожиданием чуда, мы собираемся, чтобы заверш...

2. [2025-12-26] Канал: @prekrasnie_spb
   Текст: 🗓 27 декабря 17:30 - 20:30
💳 Предновогодний 
SHADOW ECSTATIC DANCE 

Зовём тебя на праздничное танцевальное погружение, где не будет «надо». Будет тол...

3. [2025-12-23] Канал: @prekrasnie_spb
   Текст: 🗓31 декабря 20:00-3:00
💳ECSTATIC NEW YEAR — Новогодняя ночь в танце, свободе и живой магии 🙏

Мы знаем, вы ждали этого момента! Уже 7 лет подряд ECSTA...



## 4. Извлечение событий с помощью Event Miner Agent


In [8]:
# Инициализация Event Miner Agent
event_miner = EventMinerAgent(llm=client)

print("✓ EventMinerAgent инициализирован")


✓ EventMinerAgent инициализирован


In [9]:
def extract_events_from_messages(messages: List[Dict], batch_size: int = 5) -> List[Dict]:
    """
    Извлекает события из сообщений и возвращает пары сообщение-событие.
    
    Args:
        messages: Список сообщений
        batch_size: Размер батча для обработки
    
    Returns:
        Список словарей с парами сообщение-событие
    """
    results = []
    
    for i in tqdm(range(0, len(messages), batch_size), desc="Извлечение событий"):
        batch = messages[i:i + batch_size]
        
        try:
            # Извлекаем события из батча
            events = event_miner.process_messages(batch)
            
            # Сопоставляем события с сообщениями
            for j, msg in enumerate(batch):
                # Ищем событие, которое соответствует сообщению
                matching_event = None
                for event in events:
                    # Проверяем по source_message_id или по совпадению текста
                    if hasattr(event, 'source_message_id') and event.source_message_id == msg['id']:
                        matching_event = event
                        break
                    elif hasattr(event, 'original_text') and event.original_text and msg['text'][:50] in event.original_text:
                        matching_event = event
                        break
                
                # Если не нашли точное соответствие, берём по индексу (если есть)
                if not matching_event and j < len(events):
                    matching_event = events[j]
                
                results.append({
                    'message_id': msg['id'],
                    'message_date': msg['date'],
                    'message_text': msg['text'],
                    'channel': msg['channel'],
                    'event_extracted': matching_event is not None,
                    'event_title': getattr(matching_event, 'title', None) if matching_event else None,
                    'event_description': getattr(matching_event, 'description', None) if matching_event else None,
                    'event_tags': json.dumps(getattr(matching_event, 'tags', None), ensure_ascii=False) if matching_event and getattr(matching_event, 'tags', None) else None,
                    'event_location': getattr(matching_event, 'location', None) if matching_event else None,
                    'event_date': getattr(matching_event, 'date', None) if matching_event else None,
                    'event_url': getattr(matching_event, 'url', None) if matching_event else None,
                    'event_source': getattr(matching_event, 'source', None) if matching_event else None,
                    'event_json': matching_event.model_dump_json(exclude_none=True) if matching_event and hasattr(matching_event, "model_dump_json") else None
                })
                
        except Exception as e:
            # Специальная обработка для ошибки response_format schema (код 400)
            if (
                hasattr(e, "args")
                and e.args
                and isinstance(e.args[0], str)
                and "Invalid schema for response_format" in e.args[0]
            ):
                print(f"\n❌ Критическая ошибка схемы response_format (вероятно ошибка в настройке Event Miner Agent или модели):\n{e}\n\nПроверьте, что schema EventsList соответствует требованиям OpenAI - поле 'additionalProperties' должно быть указано и иметь значение false.")
                raise
            else:
                print(f"\n⚠️ Ошибка при обработке батча {i//batch_size + 1}: {e}")
                # Добавляем сообщения без извлечённых событий
                for msg in batch:
                    results.append({
                        'message_id': msg['id'],
                        'message_date': msg['date'],
                        'message_text': msg['text'],
                        'channel': msg['channel'],
                        'event_extracted': False,
                        'event_title': None,
                        'event_description': None,
                        'event_tags': None,
                        'event_location': None,
                        'event_date': None,
                        'event_url': None,
                        'event_source': None,
                        'event_json': None,
                        'error': str(e)
                    })
    
    return results

print("Функция извлечения событий готова")


Функция извлечения событий готова


In [11]:
# Извлекаем события из сообщений
print("🔄 Извлечение событий из сообщений...\n")

message_event_pairs = extract_events_from_messages(parsed_messages, batch_size=5)

# Статистика
extracted_count = sum(1 for pair in message_event_pairs if pair['event_extracted'])
print(f"\n✓ Обработано {len(message_event_pairs)} сообщений")
print(f"✓ Извлечено событий: {extracted_count} ({extracted_count/len(message_event_pairs)*100:.1f}%)")


🔄 Извлечение событий из сообщений...



Извлечение событий: 100%|██████████| 3/3 [00:51<00:00, 17.27s/it]


✓ Обработано 15 сообщений
✓ Извлечено событий: 15 (100.0%)


## 5. Сохранение результатов в CSV


In [12]:
# Создаём DataFrame и сохраняем
testing_data_path = project_root / "data" / "testing_data"
testing_data_path.mkdir(parents=True, exist_ok=True)

# Создаём DataFrame
pairs_df = pd.DataFrame(message_event_pairs)
pairs_df['extracted_at'] = datetime.now().isoformat()

# Сохраняем
pairs_file = testing_data_path / "parser_eval_pairs.csv"
pairs_df.to_csv(pairs_file, index=False, encoding="utf-8")

print(f"✓ Пары сообщение-событие сохранены в: {pairs_file}")
print(f"\nСтатистика:")
print(f"   Всего записей: {len(pairs_df)}")
print(f"   С извлечёнными событиями: {pairs_df['event_extracted'].sum()}")
print(f"   Без событий: {(~pairs_df['event_extracted']).sum()}")

pairs_df.head()


✓ Пары сообщение-событие сохранены в: /Users/andreisuhov/Desktop/llm/Journey_agent/data/testing_data/parser_eval_pairs.csv

Статистика:
   Всего записей: 15
   С извлечёнными событиями: 15
   Без событий: 0


,message_id,message_date,message_text,channel,event_extracted,event_title,event_description,event_tags,event_location,event_date,event_url,event_source,event_json,extracted_at
0,1606,2025-12-26T10:22:57+00:00,🗓 27 декабря 17:00–21:00\n💳 BIG ECSTATIC SANTA...,@prekrasnie_spb,True,BIG ECSTATIC SANTA BEAT с Vankavanka,"В самом конце декабря, когда город живёт ожида...","[""ecstatic""]",самый большой экстатичный зал Петербурга,2025-12-27,None,telegram,"{""title"":""BIG ECSTATIC SANTA BEAT с Vankavanka...",2025-12-27T01:09:29.444144
1,1605,2025-12-26T09:28:07+00:00,🗓 27 декабря 17:30 - 20:30\n💳 Предновогодний \...,@prekrasnie_spb,True,Предновогодний SHADOW ECSTATIC DANCE,Зовём тебя на праздничное танцевальное погруже...,"[""ecstatic""]",10-линия В.О. 59,2025-12-27,None,telegram,"{""title"":""Предновогодний SHADOW ECSTATIC DANCE...",2025-12-27T01:09:29.444144
2,1603,2025-12-23T16:54:51+00:00,🗓31 декабря 20:00-3:00\n💳ECSTATIC NEW YEAR — Н...,@prekrasnie_spb,True,ECSTATIC NEW YEAR,"ECSTATIC NEW YEAR — Новогодняя ночь в танце, с...","[""ecstatic""]","Livebody (Кузнечный пер.6, 3 этаж) м. Владимир...",2025-12-31,None,telegram,"{""title"":""ECSTATIC NEW YEAR"",""description"":""EC...",2025-12-27T01:09:29.444144
3,1602,2025-12-22T16:00:01+00:00,🗓 24 декабря 18:00–23:00\n💳 ECSTATIC BANYA\n\n...,@prekrasnie_spb,True,ECSTATIC BANYA,"Друзья, вы просили финальную баньку прямо пере...","[""баня""]",большая баня «Русские парные»,2025-12-24,None,telegram,"{""title"":""ECSTATIC BANYA"",""description"":""Друзь...",2025-12-27T01:09:29.444144
4,1600,2025-12-22T09:50:05+00:00,🗓27 декабря (суббота) | 17:00-21:00\n🙏ТЁПЛАЯ П...,@prekrasnie_spb,True,ТЁПЛАЯ ПРЕДНОВОГОДНЯЯ ВСТРЕЧА СООБЩЕСТВА ПРЕКР...,Любимые Прекрасные! Приближается самый тёплый ...,None,"Кафе ТРОИЦКИЙ МОСТ, наб. реки Мойки, 30",2025-12-27,None,telegram,"{""title"":""ТЁПЛАЯ ПРЕДНОВОГОДНЯЯ ВСТРЕЧА СООБЩЕ...",2025-12-27T01:09:29.444144


## 6. Агенты-оценщики для парсера

Создаём 3 агента для оценки качества извлечения событий:

### Агент 1: Оценка точности извлечения
- **Корректность извлечения** (0-10): Насколько правильно извлечены данные из сообщения
- **Полнота извлечения** (0-10): Насколько полно извлечены все доступные поля

### Агент 2: Оценка качества данных
- **Качество заголовка** (0-10): Насколько хорошо сформулирован title
- **Качество описания** (0-10): Насколько полезное и информативное описание

### Агент 3: Оценка категоризации
- **Правильность категоризации** (0-10): Корректно ли определены теги
- **Релевантность** (0-10): Действительно ли это событие или нет


In [13]:
@dataclass
class EvaluationResult:
    """Результат оценки одного агента."""
    agent_name: str
    criteria: Dict[str, int]  # {criterion_name: score}
    reasoning: str
    
    def average_score(self) -> float:
        return sum(self.criteria.values()) / len(self.criteria) if self.criteria else 0


class ParserEvaluatorAgent:
    """Агент-оценщик для парсера событий."""
    
    def __init__(self, name: str, criteria: List[str], system_prompt: str):
        self.name = name
        self.criteria = criteria
        self.system_prompt = system_prompt
        self.client = OpenAI(api_key=OPENAI_API_KEY)
    
    def evaluate(self, message_text: str, event_json: Optional[str]) -> EvaluationResult:
        """
        Оценивает качество извлечения события из сообщения.
        
        Args:
            message_text: Исходное сообщение
            event_json: JSON извлечённого события (или None если событие не извлечено)
        """
        
        criteria_str = "\n".join([f"- {c}" for c in self.criteria])
        
        event_section = ""
        if event_json:
            event_section = f"""## Извлечённое событие (JSON):
{event_json}"""
        else:
            event_section = """## Извлечённое событие:
Событие НЕ было извлечено из этого сообщения.
(Это может быть корректно, если сообщение не содержит информации о событии)"""
        
        prompt = f"""Оцени качество извлечения события из сообщения.

## Исходное сообщение:
{message_text}

{event_section}

## Критерии оценки (каждый от 0 до 10):
{criteria_str}

ВАЖНО:
- Если сообщение НЕ содержит информации о событии и событие НЕ было извлечено - это ПРАВИЛЬНО (высокие оценки)
- Если сообщение содержит событие и оно было корректно извлечено - это тоже ПРАВИЛЬНО
- Низкие оценки ставь только если:
  * Событие было в сообщении, но не извлечено
  * Событие извлечено, но данные неверные
  * Извлечено "событие" из сообщения, которое не содержит события

Верни JSON в формате:
{{
  "scores": {{
    "критерий1": число,
    "критерий2": число
  }},
  "reasoning": "краткое обоснование оценок (1-2 предложения)"
}}

Только JSON, без объяснений.
"""
        
        try:
            api_response = self.client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": self.system_prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.3,
                max_tokens=500
            )
            
            content = api_response.choices[0].message.content.strip()
            # Убираем markdown если есть
            if content.startswith("```"):
                content = content.split("```")[1]
                if content.startswith("json"):
                    content = content[4:]
            
            parsed = json.loads(content)
            
            return EvaluationResult(
                agent_name=self.name,
                criteria=parsed.get("scores", {}),
                reasoning=parsed.get("reasoning", "")
            )
        except Exception as e:
            return EvaluationResult(
                agent_name=self.name,
                criteria={c: 0 for c in self.criteria},
                reasoning=f"Ошибка оценки: {str(e)}"
            )


print("Базовый класс агента-оценщика готов")


Базовый класс агента-оценщика готов


In [14]:
# Создаём 3 агента-оценщика для парсера

agent_extraction = ParserEvaluatorAgent(
    name="Агент точности извлечения",
    criteria=[
        "Корректность извлечения",
        "Полнота извлечения"
    ],
    system_prompt="""Ты эксперт по оценке качества извлечения данных из текста.
Твоя задача - оценить, насколько правильно и полно извлечены данные о событии из сообщения.

Оценивай по шкале 0-10:
- Корректность извлечения: правильно ли извлечены title, description, date, location?
- Полнота извлечения: все ли доступные поля были заполнены?

Если сообщение не содержит события и событие не извлечено - это ХОРОШО (8-10 баллов).
Если событие было в тексте, но не извлечено - это ПЛОХО (0-3 балла)."""
)

agent_quality = ParserEvaluatorAgent(
    name="Агент качества данных",
    criteria=[
        "Качество заголовка",
        "Качество описания"
    ],
    system_prompt="""Ты эксперт по оценке качества текстовых данных.
Твоя задача - оценить качество извлечённого заголовка и описания события.

Оценивай по шкале 0-10:
- Качество заголовка: информативный, краткий, отражает суть события?
- Качество описания: полезное, содержит важную информацию, без лишнего?

Если событие не извлечено (и его не было в тексте) - ставь 5 (нейтральная оценка).
Если событие извлечено с хорошим title и description - высокие оценки."""
)

agent_categorization = ParserEvaluatorAgent(
    name="Агент категоризации",
    criteria=[
        "Правильность категоризации",
        "Релевантность"
    ],
    system_prompt="""Ты эксперт по категоризации и классификации событий.
Твоя задача - оценить правильность определения тегов и релевантность извлечённого события.

Оценивай по шкале 0-10:
- Правильность категоризации: корректно ли определены теги события?
- Релевантность: действительно ли это событие (лекция, концерт, выставка, встреча и т.д.)?

Если из обычного сообщения (не события) извлечено "событие" - низкая релевантность (0-3).
Если правильно определено, что сообщение не содержит события - высокая релевантность (8-10)."""
)

parser_agents = [agent_extraction, agent_quality, agent_categorization]

print(f"✓ Создано {len(parser_agents)} агента-оценщика:")
for agent in parser_agents:
    print(f"   - {agent.name}: {agent.criteria}")


✓ Создано 3 агента-оценщика:
   - Агент точности извлечения: ['Корректность извлечения', 'Полнота извлечения']
   - Агент качества данных: ['Качество заголовка', 'Качество описания']
   - Агент категоризации: ['Правильность категоризации', 'Релевантность']


## 7. Оценка результатов парсинга


In [15]:
def evaluate_parser_results(
    pairs_df: pd.DataFrame,
    agents: List[ParserEvaluatorAgent],
    max_samples: Optional[int] = None
) -> pd.DataFrame:
    """
    Оценивает результаты парсинга всеми агентами.
    
    Args:
        pairs_df: DataFrame с парами сообщение-событие
        agents: Список агентов-оценщиков
        max_samples: Максимальное количество записей для оценки (для экономии API)
    
    Returns:
        DataFrame с оценками
    """
    all_evaluations = []
    
    # Берём подмножество если указано
    df_to_evaluate = pairs_df.head(max_samples) if max_samples else pairs_df
    
    print(f"📊 Оценка {len(df_to_evaluate)} записей {len(agents)} агентами...\n")
    
    for idx, row in tqdm(df_to_evaluate.iterrows(), total=len(df_to_evaluate), desc="Оценка"):
        message_text = row['message_text']
        event_json = row['event_json']
        
        evaluation_row = {
            'message_id': row['message_id'],
            'channel': row['channel'],
            'message_text': message_text[:200] + '...' if len(message_text) > 200 else message_text,
            'event_extracted': row['event_extracted'],
            'event_title': row['event_title']
        }
        
        for agent in agents:
            result = agent.evaluate(message_text, event_json)
            
            for criterion, score in result.criteria.items():
                col_name = f"{agent.name}_{criterion}"
                evaluation_row[col_name] = score
            
            evaluation_row[f"{agent.name}_avg"] = result.average_score()
            evaluation_row[f"{agent.name}_reasoning"] = result.reasoning
        
        all_evaluations.append(evaluation_row)
    
    return pd.DataFrame(all_evaluations)

print("Функция оценки готова")


Функция оценки готова


In [16]:
# Запускаем оценку
# Для экономии API можно ограничить количество записей
MAX_SAMPLES_TO_EVALUATE = None  # None = все записи, или число (например, 50)

evaluations_df = evaluate_parser_results(
    pairs_df,
    parser_agents,
    max_samples=MAX_SAMPLES_TO_EVALUATE
)

print(f"\n✓ Оценено {len(evaluations_df)} записей")


📊 Оценка 15 записей 3 агентами...



Оценка: 100%|██████████| 15/15 [01:48<00:00,  7.23s/it]


✓ Оценено 15 записей


In [17]:
# Сохраняем оценки в CSV
evaluations_file = testing_data_path / "parser_eval_scores.csv"
evaluations_df.to_csv(evaluations_file, index=False, encoding="utf-8")

print(f"✓ Оценки сохранены в: {evaluations_file}")


✓ Оценки сохранены в: /Users/andreisuhov/Desktop/llm/Journey_agent/data/testing_data/parser_eval_scores.csv


## 8. Агрегация и анализ результатов


In [18]:
def aggregate_parser_scores(evaluations_df: pd.DataFrame, agents: List[ParserEvaluatorAgent]) -> Dict[str, Any]:
    """
    Агрегирует оценки по всем критериям и агентам.
    """
    results = {
        "per_agent": {},
        "per_criterion": {},
        "overall": {},
        "by_extraction_status": {}
    }
    
    all_avg_scores = []
    
    for agent in agents:
        agent_avg_col = f"{agent.name}_avg"
        if agent_avg_col in evaluations_df.columns:
            avg_score = evaluations_df[agent_avg_col].mean()
            results["per_agent"][agent.name] = {
                "average": round(avg_score, 2),
                "min": round(evaluations_df[agent_avg_col].min(), 2),
                "max": round(evaluations_df[agent_avg_col].max(), 2),
                "std": round(evaluations_df[agent_avg_col].std(), 2)
            }
            all_avg_scores.append(avg_score)
        
        for criterion in agent.criteria:
            col_name = f"{agent.name}_{criterion}"
            if col_name in evaluations_df.columns:
                results["per_criterion"][col_name] = {
                    "average": round(evaluations_df[col_name].mean(), 2),
                    "min": round(evaluations_df[col_name].min(), 2),
                    "max": round(evaluations_df[col_name].max(), 2)
                }
    
    # Общие результаты
    if all_avg_scores:
        results["overall"] = {
            "average": round(sum(all_avg_scores) / len(all_avg_scores), 2),
            "total_evaluated": len(evaluations_df),
            "events_extracted": int(evaluations_df['event_extracted'].sum()),
            "extraction_rate": round(evaluations_df['event_extracted'].mean() * 100, 1)
        }
    
    # Разбивка по статусу извлечения
    for status in [True, False]:
        subset = evaluations_df[evaluations_df['event_extracted'] == status]
        if len(subset) > 0:
            avg_scores = []
            for agent in agents:
                col = f"{agent.name}_avg"
                if col in subset.columns:
                    avg_scores.append(subset[col].mean())
            status_name = "extracted" if status else "not_extracted"
            results["by_extraction_status"][status_name] = {
                "count": len(subset),
                "average_score": round(sum(avg_scores) / len(avg_scores), 2) if avg_scores else 0
            }
    
    return results

print("Функция агрегации готова")


Функция агрегации готова


In [19]:
# Агрегируем результаты
aggregated = aggregate_parser_scores(evaluations_df, parser_agents)

print("=" * 60)
print("📊 РЕЗУЛЬТАТЫ ОЦЕНКИ ПАРСЕРА МЕРОПРИЯТИЙ")
print("=" * 60)

print(f"\n🎯 ОБЩИЙ БАЛЛ: {aggregated['overall'].get('average', 'N/A')}/10")
print(f"   Всего оценено: {aggregated['overall'].get('total_evaluated', 0)} записей")
print(f"   Извлечено событий: {aggregated['overall'].get('events_extracted', 0)}")
print(f"   Процент извлечения: {aggregated['overall'].get('extraction_rate', 0)}%")

print("\n" + "-" * 60)
print("📈 ОЦЕНКИ ПО АГЕНТАМ:")
print("-" * 60)

for agent_name, stats in aggregated['per_agent'].items():
    print(f"\n  {agent_name}:")
    print(f"    Среднее: {stats['average']}/10")
    print(f"    Мин/Макс: {stats['min']}/{stats['max']}")
    print(f"    Стд. откл.: {stats['std']}")

print("\n" + "-" * 60)
print("📋 ОЦЕНКИ ПО КРИТЕРИЯМ:")
print("-" * 60)

for criterion, stats in aggregated['per_criterion'].items():
    print(f"  {criterion}: {stats['average']}/10 (мин: {stats['min']}, макс: {stats['max']})")

print("\n" + "-" * 60)
print("📊 ОЦЕНКИ ПО СТАТУСУ ИЗВЛЕЧЕНИЯ:")
print("-" * 60)

for status, stats in aggregated['by_extraction_status'].items():
    status_label = "Событие извлечено" if status == "extracted" else "Событие не извлечено"
    print(f"  {status_label}: {stats['average_score']}/10 (записей: {stats['count']})")

print("\n" + "=" * 60)


📊 РЕЗУЛЬТАТЫ ОЦЕНКИ ПАРСЕРА МЕРОПРИЯТИЙ

🎯 ОБЩИЙ БАЛЛ: 8.78/10
   Всего оценено: 15 записей
   Извлечено событий: 15
   Процент извлечения: 100.0%

------------------------------------------------------------
📈 ОЦЕНКИ ПО АГЕНТАМ:
------------------------------------------------------------

  Агент точности извлечения:
    Среднее: 8.57/10
    Мин/Макс: 6.5/10.0
    Стд. откл.: 1.16

  Агент качества данных:
    Среднее: 8.23/10
    Мин/Макс: 7.5/8.5
    Стд. откл.: 0.46

  Агент категоризации:
    Среднее: 9.53/10
    Мин/Макс: 9.0/10.0
    Стд. откл.: 0.52

------------------------------------------------------------
📋 ОЦЕНКИ ПО КРИТЕРИЯМ:
------------------------------------------------------------
  Агент качества данных_Качество заголовка: 9.0/10 (мин: 9.0, макс: 9.0)
  Агент качества данных_Качество описания: 8.0/10 (мин: 8.0, макс: 8.0)
  Агент категоризации_Правильность категоризации: 9.83/10 (мин: 9.0, макс: 10.0)
  Агент категоризации_Релевантность: 9.83/10 (мин: 9.0, макс: 1

In [ ]:
# Визуализация результатов
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# График 1: Средние оценки по агентам
agent_names = list(aggregated['per_agent'].keys())
agent_scores = [aggregated['per_agent'][name]['average'] for name in agent_names]

colors = ['#4CAF50', '#2196F3', '#FF9800']
bars = axes[0, 0].bar(range(len(agent_names)), agent_scores, color=colors, edgecolor='black')
axes[0, 0].set_xticks(range(len(agent_names)))
axes[0, 0].set_xticklabels([n.replace('Агент ', '') for n in agent_names], rotation=15, ha='right')
axes[0, 0].set_ylim(0, 10)
axes[0, 0].set_ylabel('Средний балл')
axes[0, 0].set_title('Оценки по агентам')
axes[0, 0].axhline(y=aggregated['overall']['average'], color='red', linestyle='--', 
                   label=f"Общий: {aggregated['overall']['average']}")
axes[0, 0].legend()

for bar, score in zip(bars, agent_scores):
    axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    f'{score:.1f}', ha='center', va='bottom', fontweight='bold')

# График 2: Оценки по критериям
criteria_names = list(aggregated['per_criterion'].keys())
criteria_scores = [aggregated['per_criterion'][name]['average'] for name in criteria_names]
short_criteria = [name.split('_')[-1][:20] for name in criteria_names]

bars2 = axes[0, 1].barh(range(len(short_criteria)), criteria_scores, color='#673AB7', edgecolor='black')
axes[0, 1].set_yticks(range(len(short_criteria)))
axes[0, 1].set_yticklabels(short_criteria)
axes[0, 1].set_xlim(0, 10)
axes[0, 1].set_xlabel('Средний балл')
axes[0, 1].set_title('Оценки по критериям')

for bar, score in zip(bars2, criteria_scores):
    axes[0, 1].text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2,
                    f'{score:.1f}', va='center')

# График 3: Статистика извлечения
extraction_labels = ['Извлечено', 'Не извлечено']
extraction_counts = [
    aggregated['by_extraction_status'].get('extracted', {}).get('count', 0),
    aggregated['by_extraction_status'].get('not_extracted', {}).get('count', 0)
]
colors_pie = ['#4CAF50', '#f44336']

axes[1, 0].pie(extraction_counts, labels=extraction_labels, autopct='%1.1f%%', 
               colors=colors_pie, explode=(0.05, 0))
axes[1, 0].set_title('Статистика извлечения событий')

# График 4: Сравнение оценок по статусу извлечения
status_labels = ['Извлечено', 'Не извлечено']
status_scores = [
    aggregated['by_extraction_status'].get('extracted', {}).get('average_score', 0),
    aggregated['by_extraction_status'].get('not_extracted', {}).get('average_score', 0)
]

bars3 = axes[1, 1].bar(status_labels, status_scores, color=['#4CAF50', '#f44336'], edgecolor='black')
axes[1, 1].set_ylim(0, 10)
axes[1, 1].set_ylabel('Средний балл')
axes[1, 1].set_title('Оценки по статусу извлечения')

for bar, score in zip(bars3, status_scores):
    axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                    f'{score:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig(testing_data_path / 'parser_evaluation_chart.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ График сохранён: {testing_data_path / 'parser_evaluation_chart.png'}")


In [23]:
# Показываем лучшие и худшие результаты
avg_cols = [f"{agent.name}_avg" for agent in parser_agents]
evaluations_df['total_avg'] = evaluations_df[avg_cols].mean(axis=1)

# print("\n🏆 ТОП-3 ЛУЧШИХ РЕЗУЛЬТАТА ПАРСИНГА:")
# print("-" * 60)
# top3 = evaluations_df.nlargest(3, 'total_avg')
# for _, row in top3.iterrows():
#     print(f"\n  Сообщение: {row['message_text'][:80]}...")
#     print(f"  Событие: {row['event_title'] or 'не извлечено'}")
#     print(f"  Общий балл: {row['total_avg']:.1f}/10")

print("\n\n⚠️ ТОП-3 ХУДШИХ РЕЗУЛЬТАТА (для улучшения):")
print("-" * 60)
bottom3 = evaluations_df.nsmallest(3, 'total_avg')
for _, row in bottom3.iterrows():
    print(f"\n  Сообщение: {row['message_text']}...")
    print(f"  Событие: {row['event_title'] or 'не извлечено'}")
    print(f"  Общий балл: {row['total_avg']:.1f}/10")




⚠️ ТОП-3 ХУДШИХ РЕЗУЛЬТАТА (для улучшения):
------------------------------------------------------------

  Сообщение: 🗓 17 декабря 12:00 и 22 декабря 12:00
💳 "СПА ДЛЯ ДУШИ"

В предновогоднюю сказку и ко дню Зимнего Солнцестояния!

День и вечер звука и отдыха Состоится потрясающий Sound Healing 

В исполнении Кристины......
  Событие: СПА ДЛЯ ДУШИ
  Общий балл: 7.7/10

  Сообщение: 🗓 19-21 декабря и 26-28 декабря
💳 Увлекательное музыкальное путешествие в Карелию

Эта поездка вернёт вам силы, уйдут с головы не нужные мысли,
поднимется внутреннее настроение, придёт радость и споко......
  Событие: Увлекательное музыкальное путешествие в Карелию
  Общий балл: 8.0/10

  Сообщение: 🗓 27 декабря 17:00–21:00
💳 BIG ECSTATIC SANTA BEAT с Vankavanka 

В самом конце декабря, когда город живёт ожиданием чуда, мы собираемся, чтобы завершить год так, как умеем только мы — в танце.
BIG EC......
  Событие: BIG ECSTATIC SANTA BEAT с Vankavanka
  Общий балл: 8.3/10


## 9. Итоговый отчёт


In [21]:
# Финальный отчёт
print("\n" + "=" * 60)
print("📝 ИТОГОВЫЙ ОТЧЁТ ОЦЕНКИ ПАРСЕРА")
print("=" * 60)

print(f"""
Дата оценки: {datetime.now().strftime('%Y-%m-%d %H:%M')}

Каналы для парсинга: {len(channels)}
Сообщений спарсено: {len(parsed_messages)}
Событий извлечено: {aggregated['overall'].get('events_extracted', 0)}
Процент извлечения: {aggregated['overall'].get('extraction_rate', 0)}%

ОБЩИЙ БАЛЛ ПАРСЕРА: {aggregated['overall']['average']}/10

Детализация по агентам:
""")

for agent_name, stats in aggregated['per_agent'].items():
    print(f"  • {agent_name}: {stats['average']}/10")

print(f"""
Сохранённые файлы:
  • Пары сообщение-событие: {pairs_file}
  • Оценки: {evaluations_file}
  • График: {testing_data_path / 'parser_evaluation_chart.png'}
""")



📝 ИТОГОВЫЙ ОТЧЁТ ОЦЕНКИ ПАРСЕРА

Дата оценки: 2025-12-27 01:12

Каналы для парсинга: 1
Сообщений спарсено: 15
Событий извлечено: 15
Процент извлечения: 100.0%

ОБЩИЙ БАЛЛ ПАРСЕРА: 8.78/10

Детализация по агентам:

  • Агент точности извлечения: 8.57/10
  • Агент качества данных: 8.23/10
  • Агент категоризации: 9.53/10

Сохранённые файлы:
  • Пары сообщение-событие: /Users/andreisuhov/Desktop/llm/Journey_agent/data/testing_data/parser_eval_pairs.csv
  • Оценки: /Users/andreisuhov/Desktop/llm/Journey_agent/data/testing_data/parser_eval_scores.csv
  • График: /Users/andreisuhov/Desktop/llm/Journey_agent/data/testing_data/parser_evaluation_chart.png



In [22]:
# Показываем таблицу с результатами
display_cols = ['channel', 'message_text', 'event_extracted', 'event_title', 'total_avg']
evaluations_df[display_cols].head(10)


,channel,message_text,event_extracted,event_title,total_avg
0,@prekrasnie_spb,🗓 27 декабря 17:00–21:00\n💳 BIG ECSTATIC SANTA...,True,BIG ECSTATIC SANTA BEAT с Vankavanka,8.333333
1,@prekrasnie_spb,🗓 27 декабря 17:30 - 20:30\n💳 Предновогодний \...,True,Предновогодний SHADOW ECSTATIC DANCE,8.833333
2,@prekrasnie_spb,🗓31 декабря 20:00-3:00\n💳ECSTATIC NEW YEAR — Н...,True,ECSTATIC NEW YEAR,9.166667
3,@prekrasnie_spb,🗓 24 декабря 18:00–23:00\n💳 ECSTATIC BANYA\n\n...,True,ECSTATIC BANYA,9.166667
4,@prekrasnie_spb,🗓27 декабря (суббота) | 17:00-21:00\n🙏ТЁПЛАЯ П...,True,ТЁПЛАЯ ПРЕДНОВОГОДНЯЯ ВСТРЕЧА СООБЩЕСТВА ПРЕКР...,9.166667
5,@prekrasnie_spb,🗓 21 декабря 19:00\n💳 Творческий вечер кинореж...,True,Творческий вечер кинорежиссера Андрея Буянова ...,8.833333
6,@prekrasnie_spb,🗓21 декабря 18:00\n🙏ECSTATIC TRIBE | Празднова...,True,ECSTATIC TRIBE | Празднование Зимнего Солнцест...,9.000000
7,@prekrasnie_spb,🗓 27 декабря 17:00-21:00\n💳 BIG ECSTATIC SANTA...,True,BIG ECSTATIC SANTA BEAT с Vankavanka,8.833333
8,@prekrasnie_spb,🗓 28 декабря 18:00-21:00\n💳 Экстатичная Поэзия...,True,Экстатичная Поэзия,8.666667
9,@prekrasnie_spb,"🗓 17 декабря 12:00 и 22 декабря 12:00\n💳 ""СПА ...",True,СПА ДЛЯ ДУШИ,7.666667
